In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('method_1633_PFAS_list_step_2.csv')

In [4]:
df

,abbreviation,name,CASRN,class,smiles,pubchem_name,pubchem_cids,pubchem_smiles,similar_smiles,pubchem_n_literature,pubchem_n_transformation
0,PFOA,Perfluorooctanoic acid,335-67-1,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(...,Perfluorooctanoic acid,[9554],C(=O)(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F...,True,4639,34
1,PFOS,Perfluorooctanesulfonic acid,1763-23-1,perfluoroalkyl sulfonic acid,FC(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F...,Perfluorooctanesulfonic acid,[74483],C(C(C(C(C(F)(F)S(=O)(=O)O)(F)F)(F)F)(F)F)(C(C(...,True,3605,28
2,PFBA,Perfluorobutanoic acid,375-22-4,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(F)(F)F)(F)F)F,Perfluorobutanoic acid,[9777],C(=O)(C(C(C(F)(F)F)(F)F)(F)F)O,True,1390,14
3,PFNA,Perfluorononanoic acid,375-95-1,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F...,Perfluorononanoic acid,[67821],C(=O)(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F...,True,948,3
4,PFDA,Perfluorodecanoic acid,335-76-2,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F...,Perfluorodecanoic acid,[9555],C(=O)(C(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)...,True,797,0
5,PFHxS,Perfluorohexanesulfonic acid,355-46-4,perfluoroalkyl sulfonic acid,FC(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(S(=O)...,Perfluorohexanesulfonic acid,[67734],C(C(C(C(F)(F)S(=O)(=O)O)(F)F)(F)F)(C(C(F)(F)F)...,True,689,6
6,PFHpA,Perfluoroheptanoic acid,375-85-9,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)F,Perfluoroheptanoic acid,[67818],C(=O)(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(...,True,600,12
7,PFHxA,Perfluorohexanoic acid,307-24-4,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)F,Perfluorohexanoic acid,[67542],C(=O)(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)O,True,564,22
8,PFOSA,Perfluorooctanesulfonamide,754-91-6,perfluorooctane sulfonamide,FC(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F...,Perfluorooctanesulfonamide,[69785],C(C(C(C(C(F)(F)S(=O)(=O)N)(F)F)(F)F)(F)F)(C(C(...,True,456,20
9,PFBS,Perfluorobutanesulfonic acid,375-73-5,perfluoroalkyl sulfonic acid,FC(C(C(C(F)(F)F)(F)F)(F)F)(S(=O)(=O)O)F,Perfluorobutanesulfonic acid,[67815],C(C(C(F)(F)S(=O)(=O)O)(F)F)(C(F)(F)F)(F)F,True,443,4


In [5]:
import os
import pandas as pd
from rdkit.Chem import MolFromSmiles, AllChem, PandasTools

def clean_smiles_column(df, smiles_col, backup_col='structure_smiles_2d_qsar'):
    """Clean the SMILES column and use backup SMILES when available."""
    original_len = len(df)
    
    # Create a copy to avoid modifying the original
    df_cleaned = df.copy()
    
    # Identify rows with API errors
    error_mask = df_cleaned[smiles_col].str.contains('Error fetching data:|PUGREST', na=False)
    error_rows = df_cleaned[error_mask]
    
    print(f"Found {len(error_rows)} rows with API errors")
    
    # Replace API errors with backup SMILES where available
    replacements = 0
    for idx in error_rows.index:
        if pd.notna(df_cleaned.loc[idx, backup_col]):
            df_cleaned.loc[idx, smiles_col] = df_cleaned.loc[idx, backup_col]
            replacements += 1
    
    print(f"Replaced {replacements} API errors with backup SMILES")
    
    # Remove remaining rows with errors (those without backup SMILES)
    final_error_mask = df_cleaned[smiles_col].str.contains('Error fetching data:|PUGREST', na=False)
    df_cleaned = df_cleaned[~final_error_mask]
    
    print(f"Final number of rows after cleaning: {len(df_cleaned)}")
    
    return df_cleaned

def Featurize_Dataframe(df, SMILEScol, radius, nBits):
    original_len = len(df)
    print(f"\nStarting with {original_len} entries")
    
    # Clean the SMILES column and use backups
    df = clean_smiles_column(df, SMILEScol)
    print(f"After cleaning SMILES: {len(df)} entries")
    
    # Remove any remaining NaN values
    df = df.dropna(subset=[SMILEScol])
    print(f"After dropping NaN: {len(df)} entries")
    
    # Validate remaining SMILES strings
    valid_mols = []
    invalid_smiles = []
    
    for idx, smiles in enumerate(df[SMILEScol]):
        mol = MolFromSmiles(smiles)
        if mol is not None:
            valid_mols.append(idx)
        else:
            invalid_smiles.append((idx, smiles))
    
    df = df.iloc[valid_mols]
    
    if invalid_smiles:
        print(f"\nFound {len(invalid_smiles)} invalid SMILES strings")
        print("Sample of invalid SMILES:")
        for idx, smiles in invalid_smiles[:5]:
            print(f"Index {idx}: {smiles}")
    
    # Generate Morgan Fingerprints
    MFPs = []
    for smiles in df[SMILEScol]:
        mol = MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
        MFPs.append(fp)
    
    # Create feature matrix
    MFPsName = [f'Bit_{i}' for i in range(nBits)]
    MFPsBits = [list(l) for l in MFPs]
    
    FeaturizedDf = pd.DataFrame(MFPsBits, index=range(0, len(MFPs)), columns=MFPsName)
    FeaturizedDf = pd.concat([FeaturizedDf, df.reset_index(drop=True)], axis=1)
    
    print(f"\nFinal dataset contains {len(FeaturizedDf)} entries out of {original_len} original entries")
    
    return FeaturizedDf

# Load and process data


# Show initial data statistics
print("Initial data analysis:")
print(f"Total rows: {len(df)}")
print("\nSample of SMILES strings:")
print(df['pubchem_smiles'].head())
print("\nNumber of unique SMILES:", df['pubchem_smiles'].nunique())

# Process the data
feat = Featurize_Dataframe(df, SMILEScol='pubchem_smiles', radius=3, nBits=2048)

# Save the results
feat.to_csv('Featurized_PFAS.csv', index=False)

Initial data analysis:
Total rows: 40

Sample of SMILES strings:
0    C(=O)(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F...
1    C(C(C(C(C(F)(F)S(=O)(=O)O)(F)F)(F)F)(F)F)(C(C(...
2                       C(=O)(C(C(C(F)(F)F)(F)F)(F)F)O
3    C(=O)(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F...
4    C(=O)(C(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)...
Name: pubchem_smiles, dtype: object

Number of unique SMILES: 40

Starting with 40 entries
Found 0 rows with API errors
Replaced 0 API errors with backup SMILES
Final number of rows after cleaning: 40
After cleaning SMILES: 40 entries
After dropping NaN: 40 entries

Final dataset contains 40 entries out of 40 original entries


In [6]:
feat

,Bit_0,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,Bit_8,Bit_9,...,name,CASRN,class,smiles,pubchem_name,pubchem_cids,pubchem_smiles,similar_smiles,pubchem_n_literature,pubchem_n_transformation
0,0,0,0,0,0,0,0,0,0,0,...,Perfluorooctanoic acid,335-67-1,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(...,Perfluorooctanoic acid,[9554],C(=O)(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F...,True,4639,34
1,0,0,0,0,0,0,0,0,0,0,...,Perfluorooctanesulfonic acid,1763-23-1,perfluoroalkyl sulfonic acid,FC(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F...,Perfluorooctanesulfonic acid,[74483],C(C(C(C(C(F)(F)S(=O)(=O)O)(F)F)(F)F)(F)F)(C(C(...,True,3605,28
2,0,0,0,0,0,0,0,0,0,0,...,Perfluorobutanoic acid,375-22-4,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(F)(F)F)(F)F)F,Perfluorobutanoic acid,[9777],C(=O)(C(C(C(F)(F)F)(F)F)(F)F)O,True,1390,14
3,0,0,0,0,0,0,0,0,0,0,...,Perfluorononanoic acid,375-95-1,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F...,Perfluorononanoic acid,[67821],C(=O)(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F...,True,948,3
4,0,0,0,0,0,0,0,0,0,0,...,Perfluorodecanoic acid,335-76-2,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F...,Perfluorodecanoic acid,[9555],C(=O)(C(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)...,True,797,0
5,0,0,0,0,0,0,0,0,0,0,...,Perfluorohexanesulfonic acid,355-46-4,perfluoroalkyl sulfonic acid,FC(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(S(=O)...,Perfluorohexanesulfonic acid,[67734],C(C(C(C(F)(F)S(=O)(=O)O)(F)F)(F)F)(C(C(F)(F)F)...,True,689,6
6,0,0,0,0,0,0,0,0,0,0,...,Perfluoroheptanoic acid,375-85-9,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)F,Perfluoroheptanoic acid,[67818],C(=O)(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(...,True,600,12
7,0,0,0,0,0,0,0,0,0,0,...,Perfluorohexanoic acid,307-24-4,perfluoroalkyl carboxylic acid,FC(C(=O)O)(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)F,Perfluorohexanoic acid,[67542],C(=O)(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)O,True,564,22
8,0,0,0,0,0,0,0,0,0,0,...,Perfluorooctanesulfonamide,754-91-6,perfluorooctane sulfonamide,FC(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F...,Perfluorooctanesulfonamide,[69785],C(C(C(C(C(F)(F)S(=O)(=O)N)(F)F)(F)F)(F)F)(C(C(...,True,456,20
9,0,0,0,0,0,0,0,0,0,0,...,Perfluorobutanesulfonic acid,375-73-5,perfluoroalkyl sulfonic acid,FC(C(C(C(F)(F)F)(F)F)(F)F)(S(=O)(=O)O)F,Perfluorobutanesulfonic acid,[67815],C(C(C(F)(F)S(=O)(=O)O)(F)F)(C(F)(F)F)(F)F,True,443,4
